# Project: Performance of phenotype algorithms for the identification of opioid-exposed infants, Andrew D. Wiese et al. Hospital Pediatrics 2024
# Title: Identify birthing parents with opioid use disorder diagnosis based on ICD codes
# Summary: 
## Identify birthing parents with opioid use disorder diagnosis based on ICD codes from estimated time of conception through 30-days after delivery

# Notes:
- ICD code criteria: >2x outpatient ICD codes our >1x inpatient ICD code
- If the record had the visit_occurrence_id and visit_occurrence_id was not '0', then using visit_occurrence_id to check it in visit_occurrence, find the visit_concept_id, if it is '9201', then it was the inpatient; '9202', it was 'outpatient'.
- If there is no visit_occurrence_id, then using the visit_date



##### Algorithm steps:
```
1. Read MOM OUD diagnosis code list

2. Search condition, procedure, and observation tables for records with OUD codes
   - Join to gestational age info to filter prenatal period

3. For records with visit_occurrence_id:
   - Classify as inpatient or outpatient based on visit_concept_id

4. For records without visit_occurrence_id:
   - Classify based on visit start/end dates

5. Define groups for 1x inpatient and 2x outpatient

6. Update with records that had no visit_occurrence_id using classification rules

7. Redefine 1x inpatient and 2x outpatient groups on updated data
```

##### Data Dictionaries:

**mom_oud_code_list** - List of ICD9/10 codes indicating maternal opioid use disorder

**cond_table** - condition_occurrence table

**proc_table** - procedure_occurrence table 

**obs_table** - observation table

**visit_table** - visit_occurrence table

**mom_baby_step4_mom_oud** - Records with OUD codes linked to mom/baby info

**mom_baby_step4_mom_oud_ip1** - Distinct mom/baby pairs with 1+ inpatient OUD codes

**mom_baby_step4_mom_oud_op2** - Distinct mom/baby pairs with 2+ outpatient OUD codes 

**mom_baby_step4_mom_oud_novisit_ids** - OUD records without visit_occurrence_id

**mom_baby_step4_mom_oud_novisitoccurrenceid** - OUD records without visit_occurrence_id

**mom_baby_step4_mom_oud_novisitoccurrenceid_cond1** - Classified inpatient/outpatient using visit start/end dates

**mom_baby_step4_mom_oud_novisitoccurrenceid_cond2** - Classified as inpatient with no visit end date

**mom_baby_step4_mom_oud_novisitoccurrenceid_cond3** - Classified as outpatient with no visit end date 

**mom_baby_step4_mom_oud_update** - Combined data with visit_occurrence_id and rules-based classifications

**mom_baby_step4_mom_oud_ip1_update** - Redefined 1x inpatient group on updated data

**mom_baby_step4_mom_oud_op2_update** - Redefined 2x outpatient group on updated data

##### Usage Notes:
```
- This identifies mothers with opioid use disorder during pregnancy using diagnosis codes
- It handles cases both with and without associated visit_occurrence_id
- The outpatient definition requires 2+ codes, while inpatient only requires 1+
- Output is two distinct groups - 1x inpatient OUD and 2x outpatient OUD
```

In [0]:
%run "./project_modules"

##### MOM OUD CODEs

In [0]:
sql=f"select * from {mom_oud_code_list};"
spark.sql(sql).display()

In [0]:
def mon_records_code_list(term):
    
    if (term=="condition"):
        date_str="condition_start_date"
        datetime_str="condition_start_date"
        table_source=cond_table
    elif (term=="procedure"):
        date_str="procedure_date"
        datetime_str="procedure_datetime"
        table_source=proc_table
    elif (term=="observation"):
        date_str="observation_date"
        datetime_str="observation_date"
        table_source=obs_table
        
    sql=f"""
       select  b.*, {term}_source_value as code,{date_str} as code_date,{datetime_str} as code_datetime,visit_occurrence_id from 
       (select * from {table_source} where {term}_source_value in (select mycode from {mom_oud_code_list})) a 
       inner join 
       (select mom_person_id, baby_person_id,baby_dob,start_gestation_date,length_of_gestation 
       from global_temp.ega_w33_or_uncertain_gestation_date) b 
       on a.person_id = b.mom_person_id
       """
    
    df = spark.sql(sql)
    print("term:",term,", record count:",df.count())
    return df

##### Searching condition, procedure and observation with OUD codes

In [0]:
cond_df=mon_records_code_list('condition')
proc_df=mon_records_code_list('procedure')
obs_df=mon_records_code_list('observation')
union_dataframes([cond_df,proc_df,obs_df]).createOrReplaceTempView("merged_df")

sql=f"""
        select * from (
        select * from merged_df
        where code_date >= start_gestation_date and code_date <  date(baby_dob) + 30)
        left join 
        (select visit_occurrence_id as v_id,visit_concept_id from {visit_table}) d
        on visit_occurrence_id = d.v_id
       
    """

mom_baby_step4_mom_oud= spark.sql(sql).drop('v_id')
mom_baby_step4_mom_oud.createOrReplaceTempView("mom_baby_step4_mom_oud")

In [0]:
df_inspection("mom_baby_step4_mom_oud","all")

##### 1x inpatient (for inpatient, just need had one time OUD code)
##### select id from table_name group by id having count(*) > 0 (at least 1).

In [0]:
sql="""
      select mom_person_id,baby_person_id from 
      (select distinct * from mom_baby_step4_mom_oud where visit_concept_id = '9201')
      group by mom_person_id,baby_person_id 
      having count(*) > 0 ;
    """
spark.sql(sql).createOrReplaceTempView("mom_baby_step4_mom_oud_ip1")

##### 2x outpatient  (for outpatient, just need had two times OUD code)

In [0]:
sql="""
     select mom_person_id,baby_person_id from 
     (select distinct * from mom_baby_step4_mom_oud where visit_concept_id = '9202')
     group by mom_person_id,baby_person_id 
     having count(*) > 1 ;
    """
spark.sql(sql).createOrReplaceTempView("mom_baby_step4_mom_oud_op2")

#### Get the ID that without visit_occurrence_id and the ID did not in the ip1/op2 groups

In [0]:
sql="""
       select a.mom_person_id,a.baby_person_id from 
       (select mom_person_id,baby_person_id from mom_baby_step4_mom_oud where visit_occurrence_id is null) a 
       left join 
       (
        select mom_person_id,baby_person_id from mom_baby_step4_mom_oud_ip1 
        union 
        select mom_person_id,baby_person_id from mom_baby_step4_mom_oud_op2 
       ) b
       
       on a.mom_person_id = b.mom_person_id and a.baby_person_id=b.baby_person_id
       where b.mom_person_id is null;

    """
mom_baby_step4_mom_oud_novisit_ids= spark.sql(sql)
mom_baby_step4_mom_oud_novisit_ids.createOrReplaceTempView("mom_baby_step4_mom_oud_novisit_ids")

##### Create view that there is no visit_occurrence_id information

In [0]:
sql="""
       select a.* from mom_baby_step4_mom_oud a
       inner join 
       mom_baby_step4_mom_oud_novisit_ids b
       on a.mom_person_id = b.mom_person_id and 
       a.baby_person_id = b.baby_person_id;

    """
mom_baby_step4_mom_oud_novisitoccurrenceid= spark.sql(sql)
mom_baby_step4_mom_oud_novisitoccurrenceid.createOrReplaceTempView("mom_baby_step4_mom_oud_novisitoccurrenceid")

##### Condition1:  had visit_end_date

In [0]:
sql=f"""
       select  distinct a.mom_person_id,a.baby_person_id,a.baby_dob,a.start_gestation_date,
       a.length_of_gestation,a.code,a.code_date,a.code_datetime,b.visit_occurrence_id,b.visit_concept_id
       
       from mom_baby_step4_mom_oud_novisitoccurrenceid a
       
       inner join 
       {visit_table} b 
       on a.mom_person_id = b.person_id and visit_end_date is not null
       and visit_start_date <= code_date and code_date < visit_end_date;

    """
mom_baby_step4_mom_oud_novisitoccurrenceid_cond1= spark.sql(sql)
mom_baby_step4_mom_oud_novisitoccurrenceid_cond1.createOrReplaceTempView("mom_baby_step4_mom_oud_novisitoccurrenceid_cond1")

##### Condition2:  did not have visit_end_date, and visit concept is '9201'

In [0]:
sql=f"""
       select  distinct a.mom_person_id,a.baby_person_id,a.baby_dob,a.start_gestation_date,
       a.length_of_gestation,a.code,
       a.code_date,a.code_datetime,b.visit_occurrence_id,b.visit_concept_id 
       from mom_baby_step4_mom_oud_novisitoccurrenceid a
       left join 
       {visit_table} b 
       on a.mom_person_id = b.person_id and visit_end_date is null
       where b.visit_concept_id = '9201'
       and visit_start_date <= code_date and code_date < date_add(visit_start_date, 30);
     """
mom_baby_step4_mom_oud_novisitoccurrenceid_cond2= spark.sql(sql)
mom_baby_step4_mom_oud_novisitoccurrenceid_cond2.createOrReplaceTempView("mom_baby_step4_mom_oud_novisitoccurrenceid_cond2")

##### Condition3:  did not have visit_end_date, and visit concept is '9202'

In [0]:
sql=f"""
       select  distinct a.mom_person_id,a.baby_person_id,a.baby_dob,a.start_gestation_date,
       a.length_of_gestation,a.code,a.code_date,
       a.code_datetime,b.visit_occurrence_id,b.visit_concept_id 
       from mom_baby_step4_mom_oud_novisitoccurrenceid a
       left join 
       {visit_table} b 
       on a.mom_person_id = b.person_id and visit_end_date is null
       where b.visit_concept_id = '9202'
       and visit_start_date <= code_date and code_date < date_add(visit_start_date, 3);
     """
mom_baby_step4_mom_oud_novisitoccurrenceid_cond3= spark.sql(sql)
mom_baby_step4_mom_oud_novisitoccurrenceid_cond3.createOrReplaceTempView("mom_baby_step4_mom_oud_novisitoccurrenceid_cond3")

##### Merge mom_baby_step4_mom_oud with all 3 conditions which has been cleaned and defined if inpatient or outpatient

In [0]:
sql="""
       select * from mom_baby_step4_mom_oud
       union 
       select * from mom_baby_step4_mom_oud_novisitoccurrenceid_cond1
       union 
       select * from mom_baby_step4_mom_oud_novisitoccurrenceid_cond2
       union 
       select * from mom_baby_step4_mom_oud_novisitoccurrenceid_cond3;

    """
mom_baby_step4_mom_oud_update = spark.sql(sql)
mom_baby_step4_mom_oud_update.name='mom_baby_step4_mom_oud_update'
register_parquet_global_view(mom_baby_step4_mom_oud_update)

In [0]:
df_inspection("global_temp.mom_baby_step4_mom_oud_update","all")

##### define if 1x inpatient again

In [0]:
sql="""
     select mom_person_id,baby_person_id from 
     (select distinct * from global_temp.mom_baby_step4_mom_oud_update where visit_concept_id = '9201') 
      
     group by mom_person_id,baby_person_id 
     having count(*) > 0 ;
    """
mom_baby_step4_mom_oud_ip1_update = spark.sql(sql)
mom_baby_step4_mom_oud_ip1_update.name='mom_baby_step4_mom_oud_ip1_update'
register_parquet_global_view(mom_baby_step4_mom_oud_ip1_update)

In [0]:
df_inspection("global_temp.mom_baby_step4_mom_oud_ip1_update","all")


##### define if 2x outpatient again

In [0]:
sql="""
     select mom_person_id,baby_person_id from 
     (select distinct * from global_temp.mom_baby_step4_mom_oud_update where visit_concept_id = '9202') 
     
     group by mom_person_id,baby_person_id 
     having count(*) > 1 ;
    """
mom_baby_step4_mom_oud_op2_update = spark.sql(sql)
mom_baby_step4_mom_oud_op2_update.name='mom_baby_step4_mom_oud_op2_update'
register_parquet_global_view(mom_baby_step4_mom_oud_op2_update)

In [0]:
df_inspection("global_temp.mom_baby_step4_mom_oud_op2_update","all")